In [1]:
import numpy as np
import math
from Genetic import GA
from scipy.optimize import minimize, Bounds
from scipy.optimize import OptimizeResult
import matplotlib.pyplot as plt
import sys
import time

# Определение функций

In [2]:
# Функция Растригина
def rastrigin_function(X):

    dim=len(X)         

    OF=0
    for i in range (0,dim):
        OF+=(X[i]**2)-10*math.cos(2*math.pi*X[i])+10

    return OF

def sphere(X):
    return sum(x**2)

rastrigin_varbound=np.array([[-5.12,5.12]]*2)

In [3]:
from statistics import mean
from statistics import variance

def f1(X):
    res = rastrigin_function(X)
    return res 

In [7]:
def progress(count, total, status=''):
    bar_len = 50
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '|' * filled_len + '_' * (bar_len - filled_len)

    sys.stdout.write('\r%s %s%s/%s %s' % (status, bar, count, total, 'итераций'))
    sys.stdout.flush()  

In [4]:
xs = [[0,0],[0,1],[0.2,0.2],[0.5,0],[0.5,0.5],[0.1,0.1]]

# Функция, которая будет вызываться при каждой итерации
def callbackF1(X):    
    function_results.append(f1(X))

In [9]:
xs = []
for i in range(5, 55, 5):
    for y in range(5, 55, 5):
        xs.append([i / 100.00, y / 100.00])

total_runs = len(xs)
values = []
progress(0, total_runs, "Прогресс: ")

for i in range(total_runs):
    function_results = []
    model = minimize(f1, xs[i], method='SLSQP', bounds=rastrigin_varbound, callback=callbackF1)
    values.append(function_results[-1])
    progress(i + 1, total_runs, "Прогресс: ")    

print("\nМатематическое ожидание: ", mean(values))
print("Дисперсия: ", variance(values))

Прогресс:  ||||||||||||||||||||||||||||||||||||||||||||||||||100/100 итераций
Математическое ожидание:  1.8721204119018446
Дисперсия:  3.5114596092066215


In [10]:
algorithm_param = {'max_num_iteration': 70,
                   'population_size': 70,
                   'mutation_probability': 0.4,
                   'elit_ratio': 0.01,
                   'crossover_probability': 0.7,
                   'parents_portion': 0.3,
                   'crossover_type': 'uniform',
                   'max_iteration_without_improv': None}

model=GA(function=f1,
         dimension=2,
         variable_type='real',
         variable_boundaries=rastrigin_varbound,
         progress_bar=False, 
         convergence_curve=False,
         algorithm_parameters=algorithm_param)

total_runs = 100
values = []
progress(1, total_runs, "Прогресс: ")

for i in range(total_runs):    
    model.run()
    values.append(model.best_function)    
    progress(i + 1, total_runs, "Прогресс: ")
    
print("\nЗначение математического ожидания: ", mean(values))   
print("Значение дисперсии: ", variance(values))

Прогресс:  ||||||||||||||||||||||||||||||||||||||||||||||||||100/100 итераций
Значение математического ожидания:  0.009956033563179804
Значение дисперсии:  0.0003095164392576652


### Определим время, требуемое для получения априори известного значения глобального экстремума оптимизируемой функции

In [30]:
# Функция для поиска априори известного значения
def f2(X):
    res = rastrigin_function(X)
    if res < 0.001:
        res = 0
    return res

К сожалению, GA не находит точное значение экстремума (значение имеет погрешность около 0,01), поэтому в функции f2 априорное значение определено как промежуток вещественных чисел от 0 до 0,01

In [33]:
algorithm_param = {'max_num_iteration': 150,
                   'population_size': 100,
                   'mutation_probability': 0.2,
                   'elit_ratio': 0.01,
                   'crossover_probability': 0.3,
                   'parents_portion': 0.4,
                   'crossover_type': 'uniform',
                   'max_iteration_without_improv': 2}

model=GA(function=f2,
         dimension=2,
         variable_type='real',
         variable_boundaries=rastrigin_varbound,
         progress_bar=False, 
         convergence_curve=False,
         algorithm_parameters=algorithm_param)
result = 1
while int(result) != 0:
    start = time.time()
    model.run()
    stop = time.time()
    result = model.best_function
print("\n\nВремя: ",stop - start)



Время:  2.883575916290283


In [32]:
result = 1
while int(result) != 0:
    is_zero = False
    start = time.time()
    model = minimize(f2, xs[i], method='SLSQP', bounds=rastrigin_varbound, callback=callbackF1)
    stop = time.time()
    result = model.fun
print("Время: ",stop - start)

Время:  0.0034661293029785156


### Оценить время нахождения последнего (возможно, локального) экстремума функции соответствия (целевой функции) 

In [62]:
class Params:
    def reset():
        Params.result = -1
        Params.f3_stop = time.time()
        Params.generation = 0
        Params.current_generation = 0
        
    def save(res, stop, cur_gen):
        Params.result = res
        Params.f3_stop = stop
        Params.current_generation = cur_gen 
        
    def save_generation(gen):
        Params.generation = gen
        
    def get_generation():
        return Params.generation
    
    def get_result():
        return Params.result
    
    def get_stop():
        return Params.f3_stop
    
    def get_current_generation():
        return Params.current_generation
    
Params.reset = staticmethod(Params.reset)
Params.save = staticmethod(Params.save)
Params.save_generation = staticmethod(Params.save_generation)
Params.get_generation = staticmethod(Params.get_generation)
Params.get_result = staticmethod(Params.get_result)
Params.get_stop = staticmethod(Params.get_stop)
Params.get_current_generation = staticmethod(Params.get_current_generation)

def f3(X):    
    res = rastrigin_function(X)
    Params.save_generation(Params.get_generation() + 1)
    if Params.get_result() < 0 or res < Params.get_result():
        Params.save(res, time.time(), Params.get_generation())
    return res

In [61]:
algorithm_param = {'max_num_iteration': 150,
                   'population_size': 100,
                   'mutation_probability': 0.2,
                   'elit_ratio': 0.01,
                   'crossover_probability': 0.3,
                   'parents_portion': 0.4,
                   'crossover_type': 'uniform',
                   'max_iteration_without_improv': None}


model=GA(function=f3,
         dimension=2,
         variable_type='real',
         variable_boundaries=rastrigin_varbound,
         progress_bar=True, 
         convergence_curve=False,
         algorithm_parameters=algorithm_param)

Params.reset()
start = time.time()
model.run()

print("\nЭкстремум: ", Params.get_result())
print("Время: ", Params.get_stop() - start)
print("Поколение: ", Params.get_current_generation())

                                                                                                    
Экстремум:  2.1682284371493665e-05
Время:  8.907960891723633
Поколение:  8986
